In [1]:
from pymongo import MongoClient


client = MongoClient('localhost', 27017)
db = client['NBA']

In [12]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
db.list_collection_names()

['historical_raptor',
 'player2k',
 'shot_chart',
 'game_log',
 'historical_pbp_modelled',
 'historical_pbp']

In [4]:
db.historical_pbp_modelled.find_one({'OT_ind': 1})

{'_id': ObjectId('61e9f082f0ee3d9953e248d1'),
 'GAME_ID': 21500022,
 'home_poss': 0,
 'diff': 0,
 'time_remaining': 300,
 'OT_ind': 1,
 'elo1_pre': 1357.17448039684,
 'elo2_pre': 1571.49824412864,
 'home_team_win': 0.0,
 'MATCHUP': 'ORL vs. OKC',
 'preds_w_elo': 0.42507651448249817,
 'preds_wO_elo': 0.5649775862693787}

In [40]:
g = pd.DataFrame.from_records(db.historical_pbp_modelled.find({'GAME_ID': 21500022}))

In [24]:
game_info = pd.DataFrame.from_records(db.game_log.find({'GAME_ID': 21500022}))

In [73]:
game_info

,_id,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,...,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,Home,Away,home_team_win
0,61e9eb6cf0ee3d9953abc9e2,22015,1610612753,ORL,Orlando Magic,21500022,2015-10-30,ORL vs. OKC,L,290,...,10,10,18,31,136,-3,1,ORL,OKC,0.0


In [14]:
def plot_game(df, game_info):
    # Create traces
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = df['time_remaining'][::-1], y=df['preds_w_elo'],
                        mode='lines', name='Win prob using elo'))

    fig.add_trace(go.Scatter(x = df['time_remaining'][::-1], y=df['preds_wO_elo'],
                        mode='lines', name='Win prob'))

    fig.add_vline(x=720, line_width=3, line_dash="dash", line_color="green", name='Q2', annotation_text='Q2')
    fig.add_vline(x=1440, line_width=3, line_dash="dash", line_color="green", name='Q3', annotation_text='Q3')
    fig.add_vline(x=2160, line_width=3, line_dash="dash", line_color="green", name='Q4', annotation_text='Q4')

    fig.update_layout(title=f"{game_info['Home'].iloc[0]} win probability against{game_info['Away'].iloc[0]}",
                   xaxis_title='Time Remaining',
                   yaxis_title='Probability')


    return fig

In [15]:
plot_game(g, game_info)